<a href="https://colab.research.google.com/github/OmarMohammed88/NLP_Tasks/blob/main/AllDataSets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##weights and biases

In [ ]:
!pip install --upgrade wandb

     |████████████████████████████████| 1.4MB 5.5MB/s 
     |████████████████████████████████| 102kB 12.1MB/s 
     |████████████████████████████████| 102kB 12.4MB/s 
     |████████████████████████████████| 122kB 30.3MB/s 
     |████████████████████████████████| 163kB 28.9MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=98e996d0c8ed2983f0c62417471c3a87f44a76d8afd4f781db0785f83245fab9
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=130f78f5528d9eeb93258956c8c611c7f5ddc1772aaf52e13c53576de8bb4214
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=90

In [ ]:
!wandb login d3214840db57c8ceb248f2168774d9d01ddd1d3c

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


##imports

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn import metrics
from pprint import pprint
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import names
import nltk
nltk.download('names')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re 
from nltk.corpus import stopwords
#from sklearn.datasets import fetch_rcv1


[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h1>First DataSets</h1>

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
def twenty_newsgroup_to_csv(data,name):
    newsgroups_train = fetch_20newsgroups(subset= data , remove=('headers', 'footers', 'quotes'))

    df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
    df.columns = ['text', 'target']

    targets = pd.DataFrame( newsgroups_train.target_names)
    targets.columns=['title']

    out = pd.merge(df, targets, left_on='target', right_index=True)
    out['date'] = pd.to_datetime('now')
    out.to_csv(name+'.csv')

twenty_newsgroup_to_csv('train','train')
twenty_newsgroup_to_csv('test','test')

In [ ]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [ ]:
df.dropna(subset = ["text"], inplace=True)
df_test.dropna(subset = ["text"], inplace=True)

In [ ]:
all_names = names.words()
WNL = WordNetLemmatizer()
def clean(data):
    cleaned = defaultdict(list)
    count = 0
    for group in data:
        for words in group.split():
            if words.isalpha() and words not in all_names:
                cleaned[count].append(WNL.lemmatize(words.lower()))
        cleaned[count] = ' '.join(cleaned[count])
        count +=1 
    return(list(cleaned.values()))

In [ ]:
df['text'] = clean(df['text'])
df_test['text'] = clean(df_test['text'])

In [ ]:
print(len(df))
print(len(df_test))


11096
7370


In [ ]:
df.drop(labels=['target','date'],axis=1,inplace=True)


In [ ]:
df_test.drop(labels=['target','date'],axis=1,inplace=True)


<h1>Second Dataset</h1>

In [ ]:
nltk.download('reuters')
nltk.download('punkt')
from nltk.corpus import stopwords, reuters

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
documents = reuters.fileids()
 
train_docs_id = list(filter(lambda doc: doc.startswith("train"),
                            documents))
test_docs_id = list(filter(lambda doc: doc.startswith("test"),
                           documents))
 
train_docs = [reuters.raw(doc_id) for doc_id in train_docs_id]
test_docs = [reuters.raw(doc_id) for doc_id in test_docs_id]

In [ ]:
docs_pdf_train = pd.DataFrame(train_docs) 
docs_pdf_test = pd.DataFrame(test_docs) 
docs_pdf_train['text'] = pd.DataFrame(train_docs) 
docs_pdf_test['text'] = pd.DataFrame(test_docs) 
docs_pdf_train.drop([docs_pdf_train.columns[0]], axis ='columns',inplace=True)
docs_pdf_test.drop([docs_pdf_test.columns[0]], axis ='columns',inplace=True)
docs_pdf_train.dropna(inplace=True)
docs_pdf_test.dropna(inplace=True)

In [ ]:
docs_pdf_train.dropna(subset = ["text"], inplace=True)
docs_pdf_train.replace(r'-', ' ', regex=True,inplace=True)
docs_pdf_train.replace(r'\t', ' ', regex=True,inplace=True)
docs_pdf_train.replace(r'\n', ' ', regex=True,inplace=True)
docs_pdf_train.replace(r'   ', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'-', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'\t', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'\n', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'   ', ' ', regex=True,inplace=True)

In [ ]:
docs_pdf_train['text'] = clean(docs_pdf_train['text'])
docs_pdf_test['text'] = clean(docs_pdf_test['text'])

In [ ]:
train_labels = [reuters.categories(doc_id)for doc_id in train_docs_id]
test_labels = [reuters.categories(doc_id)for doc_id in test_docs_id]

In [ ]:
achedStopWords = stopwords.words("english")
 
def tokenize(text):
  min_length = 3
  words = map(lambda word: word.lower(), word_tokenize(text))
  words = [word for word in words if word not in achedStopWords]
  tokens = (list(map(lambda token: PorterStemmer().stem(token),
                                   words)))
  p = re.compile('[a-zA-Z]+');
  filtered_tokens =list(filter (lambda token: p.match(token) and
                               len(token) >= min_length,
                               tokens))
  return filtered_tokens

def word_tokenizer():
  tokenize_docs = []
  docs_tokens = docs_pdf_train['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs

In [ ]:
tokenize_docs = word_tokenizer()

In [ ]:
def word_tokenizer_test():
  tokenize_docs = []
  docs_tokens = docs_pdf_test['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs

In [ ]:
tokenize_docs_test = word_tokenizer_test()

In [ ]:
df_train_2=pd.DataFrame(docs_pdf_train,columns=['text'])
df_train_2['target']=''
df_train_2['target']=train_labels


In [ ]:
df_test_2=pd.DataFrame(docs_pdf_test,columns=['text'])
df_test_2['target']=''
df_test_2['target']=test_labels
df_test_2

,text,target
0,asian exporter fear damage from japan rift mou...,[trade]
1,china daily say vermin eat pct grain stock a s...,[grain]
2,japan to revise long term energy demand downwa...,"[crude, nat-gas]"
3,thai trade deficit widens in first quarter tra...,"[corn, grain, rice, rubber, sugar, tin, trade]"
4,indonesia see cpo price rising sharply indones...,"[palm-oil, veg-oil]"
...,...,...
3014,chase corp make offer for entregrowth corp ltd...,[acq]
3015,tokyo dealer see dollar poised to breach yen f...,"[dlr, money-fx, yen]"
3016,conference cut gulf war risk charge the pakist...,[ship]
3017,soviet industrial slower in the soviet industr...,[ipi]


In [ ]:
df_train_2.rename(columns={'target':'title'},inplace=True)
df_test_2.rename(columns={'target':'title'},inplace=True)


In [ ]:
frames_train=[df,df_train_2]
frames_test=[df_test,df_test_2]
all_dataframes_train=pd.concat(frames_train,keys=['x','y'])
all_dataframes_test=pd.concat(frames_test,keys=['x','y'])


In [ ]:
all_dataframes_train.drop(labels='Unnamed: 0',inplace=True,axis=1)
all_dataframes_test.drop(labels='Unnamed: 0',inplace=True,axis=1)

##Bert

In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project="gru-bert")

In [ ]:
!pip install 'kashgari>=2.0.0a0'	

In [ ]:
import logging
from kashgari.embeddings import BertEmbedding
from kashgari.tasks.classification import BiGRU_Model
from kashgari.tasks.classification import BiLSTM_Model


In [ ]:
logging.basicConfig(level='DEBUG')


In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2020-09-01 16:28:47--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip.1’

uncased_L-12_H-768_ 100%[===================>] 388.84M   191MB/s    in 2.0s    

2020-09-01 16:28:49 (191 MB/s) - ‘uncased_L-12_H-768_A-12.zip.1’ saved [407727028/407727028]



In [ ]:
!unzip '/content/uncased_L-12_H-768_A-12.zip'

Archive:  /content/uncased_L-12_H-768_A-12.zip
replace uncased_L-12_H-768_A-12/bert_model.ckpt.meta? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
bert_embed = BertEmbedding('/content/uncased_L-12_H-768_A-12')

2020-09-01 17:48:14,596 [DEBUG] kashgari - ------------------------------------------------
2020-09-01 17:48:14,596 [DEBUG] kashgari - Loaded transformer model's vocab
2020-09-01 17:48:14,599 [DEBUG] kashgari - config_path       : /content/uncased_L-12_H-768_A-12/bert_config.json
2020-09-01 17:48:14,600 [DEBUG] kashgari - vocab_path      : /content/uncased_L-12_H-768_A-12/vocab.txt
2020-09-01 17:48:14,601 [DEBUG] kashgari - checkpoint_path : /content/uncased_L-12_H-768_A-12/bert_model.ckpt
2020-09-01 17:48:14,603 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31

In [ ]:
model = BiLSTM_Model(bert_embed, sequence_length=400, multi_label=True)

In [ ]:
toknize_train=all_dataframes_train['text'].tolist()
toknize_train=[i.split(' ') for i in toknize_train]

In [ ]:
modifed_list_labels=all_dataframes_train['title'][:11096].tolist()
modifed_list_labels=[[i] for i in modifed_list_labels]


In [ ]:
all_dataframes_train['title'][:11096]=modifed_list_labels
train_labels_=all_dataframes_train['title'].tolist()


In [ ]:
toknize_test=all_dataframes_test['text'].tolist()
toknize_test=[i.split(' ') for i in toknize_test]


In [ ]:
modifed_list_labels_test=all_dataframes_test['title'][:7370].tolist()
modifed_list_labels_test=[[i] for i in modifed_list_labels_test]
all_dataframes_test['title'][:7370]=modifed_list_labels_test
test_labels_=all_dataframes_test['title'].tolist()


In [ ]:
model.fit(toknize_train,train_labels_,toknize_test,test_labels_,epochs=35)

In [ ]:
model.save("LSTM")

In [ ]:
!mkdir '/content/LSTM' 
!cp  '/content/drive/My Drive/Model_weights/embed_model_weights.h5' '/content/LSTM'
!cp  '/content/drive/My Drive/Model_weights/model_config.json' '/content/LSTM'
!cp  '/content/drive/My Drive/Model_weights/model_weights.h5' '/content/LSTM'


In [ ]:

loaded_model = BiLSTM_Model.load_model('/content/LSTM/')

2020-09-01 17:51:13,866 [DEBUG] kashgari - ------------------------------------------------
2020-09-01 17:51:13,867 [DEBUG] kashgari - Loaded transformer model's vocab
2020-09-01 17:51:13,868 [DEBUG] kashgari - config_path       : /content/uncased_L-12_H-768_A-12/bert_config.json
2020-09-01 17:51:13,870 [DEBUG] kashgari - vocab_path      : /content/uncased_L-12_H-768_A-12/vocab.txt
2020-09-01 17:51:13,871 [DEBUG] kashgari - checkpoint_path : /content/uncased_L-12_H-768_A-12/bert_model.ckpt
2020-09-01 17:51:13,872 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31

In [ ]:
def predict(docs):
  tokenize_docs = []
  df=pd.DataFrame([docs],columns=['text'])
  docs_tokens = df['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs

In [ ]:
loaded_model.predict([nltk_tokens])

2020-09-01 18:08:44,835 [DEBUG] kashgari - predict input shape (2, 1, 62) x: 
(array([[  101,   100, 13058,   100,  5658,   100, 14931,  1058, 14931,
         5658,  1058,  5096,   100,  1058,   100,   100,   100,   100,
         1058,   100,  3157,   100,   100, 14931,  1058, 14931,  5658,
          100,  1058,   100,  5096,   100,  1058,   100,   100,   100,
          100,  1058,   100,   100,  2566,  3745, 11138,   100,  1997,
         2048,   100,  3745,  1999,  3988,  2270,  3157,   100,  2765,
         2421,  9313,  5114,  1997,   100,  2030, 14931,   102]],
      dtype=int32), array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32))
2020-09-01 18:08:44,907 [DEBUG] kashgari - predict output shape (1, 110)


[['earn']]

In [ ]:
loaded_model.evaluate(toknize_test,test_labels_)

2020-09-01 18:10:17,533 [DEBUG] kashgari - predict input shape (2, 10389, 512) x: 
(array([[ 101, 1045, 2572, ...,    0,    0,    0],
       [ 101, 1045, 2031, ...,    0,    0,    0],
       [ 101, 1045, 2342, ...,    0,    0,    0],
       ...,
       [ 101, 3034, 3013, ...,    0,    0,    0],
       [ 101, 3354, 3919, ...,    0,    0,    0],
       [ 101, 2416, 2730, ...,    0,    0,    0]], dtype=int32), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32))
2020-09-01 18:17:21,525 [DEBUG] kashgari - predict output shape (10389, 110)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support
                earn     0.9709    0.9825    0.9767      1087
                 acq     0.9677    0.9583    0.9630       719
      comp.windows.x     0.4296    0.5964    0.4995       394
    rec.sport.hockey     0.9121    0.7143    0.8011       392
soc.religion.christian     0.5245    0.8010    0.6339       387
     rec.motorcycles     0.9403    0.3214    0.4791       392
comp.sys.ibm.pc.hardware     0.5000    0.0052    0.0102       387
        misc.forsale     0.8287    0.6108    0.7033       388
             sci.med     0.9623    0.5312    0.6846       384
  rec.sport.baseball     0.7622    0.6909    0.7248       385
           sci.crypt     0.4503    0.6518    0.5326       382
           sci.space     0.9820    0.4282    0.5964       383
     sci.electronics     0.5520    0.3603    0.4360       383
comp.os.ms-windows.misc     1.0000    0.0026    0.0052       382
       comp.graphics     0.7683    0.1641    0.2704       384

{'detail': {'acq': {'f1': 0.962962962962963,
   'precision': 0.9676966292134831,
   'recall': 0.9582753824756607,
   'support': 719},
  'alt.atheism': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 313},
  'alum': {'f1': 0.5882352941176471,
   'precision': 0.9090909090909091,
   'recall': 0.43478260869565216,
   'support': 23},
  'barley': {'f1': 0.45454545454545453,
   'precision': 0.625,
   'recall': 0.35714285714285715,
   'support': 14},
  'bop': {'f1': 0.5199999999999999,
   'precision': 0.65,
   'recall': 0.43333333333333335,
   'support': 30},
  'carcass': {'f1': 0.6250000000000001,
   'precision': 0.7142857142857143,
   'recall': 0.5555555555555556,
   'support': 18},
  'castor-oil': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 1},
  'cocoa': {'f1': 0.9714285714285714,
   'precision': 1.0,
   'recall': 0.9444444444444444,
   'support': 18},
  'coconut': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 2},
  'coconut-oil': {'f1': 0.0, 'precision': 0.0,

In [ ]:
all_dataframes_test['title'][10360]

['money-fx']

In [ ]:
nltk_tokens = nltk.word_tokenize(text)


In [ ]:
x=all_dataframes_test['text'][10360]


In [ ]:
text=all_dataframes_test['text'][10001]

In [ ]:
text

'jepson corp qtr net shr ct v ct net v sale mln v mln avg shrs mln v mln nine mths shr ct v ct net mln v mln sale mln v mln avg shrs mln v mln qtr per share reflects issuance of two mln share in initial public nine mth result include extraordinary gain of dlrs or ct'

In [ ]:
all_dataframes_train.to_csv("train_allDatasets.csv")
all_dataframes_test.to_csv("test_allDatasets.csv")


In [ ]:
!cp '/content/train_allDatasets.csv' '/content/drive/My Drive/'
!cp '/content/test_allDatasets.csv' '/content/drive/My Drive/'